## Homework: Vector Search
### Author: Glen Lopez

### Embeddings
### Q1. Embedding the query

In [1]:
from fastembed import TextEmbedding
import numpy as np

In [2]:
### Embed the query: 'I just discovered the course. Can I join now?'. Use the 'jinaai/jina-embeddings-v2-small-en' model.
model = TextEmbedding('jinaai/jina-embeddings-v2-small-en')
query = 'I just discovered the course. Can I join now?'

In [4]:
# Embed the query
embedding = list(model.embed(query))[0]

In [6]:
min_value = min(embedding)

In [10]:
min_value

np.float64(-0.11726373551188797)

In [18]:
if isinstance(embedding, np.ndarray):
    print('Is a NumPy array. And the dim is:', len(embedding))

Is a NumPy array. And the dim is: 512


#### Cosine similarity

In [21]:
### Check if the vector is already normalized: their length is 1.0
np.linalg.norm(embedding)

np.float64(1.0)

In [22]:
### Compute the cosine of the query vector with itself
embedding.dot(embedding)

np.float64(1.0)

### Q2. Cosine similarity with another vector

In [23]:
### Alright, let's embed this document.

In [24]:
doc = 'Can I still join the course after the start date?'

In [25]:
embedding_doc = list(model.embed(doc))[0]

In [33]:
if isinstance(embedding_doc, np.ndarray):
    print('Is a NumPy array. And the dim is:', len(embedding_doc))

Is a NumPy array. And the dim is: 512


In [29]:
### Check if the vector is already normalized: their length is 1.0
np.linalg.norm(embedding_doc)

np.float64(0.9999999999999999)

In [31]:
### question: What's the cosine similarity between the vector for the query and the vector for the document?
embedding.dot(embedding_doc)

np.float64(0.9008528856818037)

### Q3. Ranking by cosine

In [32]:
### We use the follow document for the nex questions
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [72]:
### Compute the embeddings for the text field, and compute the cosine between the query vector and all the documents.
documents_t = [item["text"] for item in documents]

In [35]:
documents_t

["Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
 "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'You can start by instal

In [36]:
embeddings_docs = list(model.embed(documents_t))

In [38]:
len(embeddings_docs)

5

In [39]:
len(embeddings_docs[0])

512

In [91]:
### In this step we using the Hint: if you put all the embeddings of the text field in one matrix V (a single 2-dimensional numpy array), then computing the cosine becomes a matrix multiplication:
V = np.array(embeddings_docs)

In [92]:
high_similarity = V.dot(embedding)

In [93]:
### What's the document index with the highest similarity? (Indexing starts from 0):
idx_max_value = np.argmax(high_similarity)
max_value = high_similarity[idx_max_value]

In [94]:
print('document_index', idx_max_value, 'value:', max_value)

document_index 1 value: 0.8182378361919105


### Q4. Ranking by cosine, version two

In [82]:
full_text = [item['question'] + ' ' + item["text"] for item in documents]

In [83]:
full_text

["Course - Can I still join the course after the start date? Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'Course - Can I follow the course after it finishes? Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
 "Course - When will the course start? The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin t

In [84]:
embeddings_full_text = list(model.embed(full_text))

In [89]:
V_full = np.array(embeddings_full_text)

In [95]:
high_similarity_f = V_full.dot(embedding)

In [97]:
### What's the document index with the highest similarity? (Indexing starts from 0):
idx_max_value_f = np.argmax(high_similarity_f)
max_value_f = high_similarity_f[idx_max_value_f]

In [99]:
print('document_index_f', idx_max_value_f, 'value_f:', max_value_f)

document_index_f 0 value_f: 0.8514542983476502


#### Is it different from Q3? If yes, why?
##### Yes, because Q3 uses only the answer text, while Q4 combines the question and the answer, which provides more context. This added context improves the quality of the embeddings and results in more accurate cosine similarity scores.

### Q5. Selecting the embedding model

In [107]:
import json

EMBEDDING_DIMENSIONALITY = 384

for model in TextEmbedding.list_supported_models():
    if model["dim"] == EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-en",
  "sources": {
    "hf": "Qdrant/bge-small-en",
    "url": "https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.13,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "BAAI/bge-small-en-v1.5",
  "sources": {
    "hf": "qdrant/bge-small-en-v1.5-onnx-q",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.067,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "snowflake/snowflake-arctic-embed-xs",
  "sou

In [108]:
model = 'BAAI/bge-small-en'

### Q6. Indexing with qdrant

In [5]:
import requests 
from qdrant_client import QdrantClient, models

In [4]:
qd_client = QdrantClient("http://localhost:6333")

In [6]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [9]:
model = 'BAAI/bge-small-en'
EMBEDDING_DIMENSIONALITY = 384

In [10]:
collection_name = "qa_mlzoomcamp"

In [16]:
query = 'I just discovered the course. Can I join now?'

In [11]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [12]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [14]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [15]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [20]:
def search(query, limit=1):

    results = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [21]:
search(query)

QueryResponse(points=[ScoredPoint(id=14, version=2, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)])